In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.basics import *

In [3]:
import dask.dataframe as dd

In [4]:
# df = pd.read_feather('tmp/train_small.feather')
df_l = dd.read_parquet('tmp/train_labeled.parquet')
fn = 'tmp/train_feat.parquet'

In [5]:
# df = dd.read_parquet('tmp/test_labeled.parquet')
# fn = 'tmp/test_feat.parquet'

In [6]:
# convert everything to smallest data type
# df.max()

In [7]:
# df = df.astype(dtype={'Semana': np.uint8,
#                       'Canal_ID': np.uint8,
#                       'Ruta_SAK': np.uint16,
#                       'Venta_uni_hoy': np.uint16,
#                       'Venta_hoy': np.float32,
#                       'Dev_uni_proxima': np.uint16,
#                       'Dev_proxima': np.float32,
#                       'Demanda_uni_equil': np.uint16,
#                       'geo_lbl': np.uint16,
#                       'client_lbl': np.uint32,
#                       'product_lbl': np.uint16})
# df = df.to_sparse()

In [8]:
key_cols = ['client_lbl', 'product_lbl', 'geo_lbl', 'Semana']
data_cols = ['Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima', 'sem_diff']
other_cols = ['Demanda_uni_equil', 'Canal_ID', 'Ruta_SAK']

In [9]:
df_r = df_l.copy()

df_r = df_r.assign(sem_diff = 1)
df_r = df_r[key_cols+data_cols]

df_r['Semana'] = df_r['Semana'] + 1
df_r = df_r[df_r['Semana'] < 10]

In [10]:
# ind_l = df_l['Semana'] > 3
# ind_r = df_r['Semana'] < 10

df_l = df_l[other_cols+key_cols]
df_l = df_l[df_l['Semana'] > 4] 

df = (df_l.merge(df_r, 
             how='left',
             on=key_cols, 
             suffixes=('_0', '_1')))

In [11]:
df_r['Semana'] = df_r['Semana'] + 1
df_r['sem_diff'] = 2
df_r = df_r[df_r['Semana'] < 10]

In [12]:
# ind_l = df_l['Semana'] > 4
# ind_r = df_r['Semana'] < 10

df_l = df_l[df_l['Semana'] > 5] 
df = df.append(df_l.merge(df_r, 
                      how='left',
                      on=key_cols, 
                      suffixes=('_0', '_1')))

In [ ]:
df = df.compute()
df.to_parquet(fn)

## test

In [ ]:
df_t = dd.read_parquet('tmp/test_labeled.parquet')
fn = 'tmp/test_feat.parquet'

In [ ]:
key_cols = ['client_lbl', 'product_lbl', 'geo_lbl', 'Semana']
data_cols = ['Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima', 'sem_diff']
other_cols = ['Canal_ID', 'Ruta_SAK']

In [ ]:
df_r = dd.read_parquet('tmp/train_labeled.parquet')

df_r = df_r.assign(sem_diff = 1)
df_r = df_r[key_cols+data_cols]

df_r = df_r[df_r['Semana'] == 9]
df_r['Semana'] = df_r['Semana'] + 1

In [ ]:
# ind_l = df_l['Semana'] == 10

df_l = df_t.loc[df_l['Semana'] == 10, other_cols+key_cols]

df = (df_l
      .merge(df_r, 
             how='left',
             on=key_cols, 
             suffixes=('_0', '_1')))

In [ ]:
df_r['Semana'] = df_r['Semana'] + 1
df_r['sem_diff'] = 2

In [ ]:
# ind_l = df_l['Semana'] == 11

df_l = df_t.loc[df['Semana'] == 11, other_cols+key_cols]

df = df.append(df_l
               .merge(df_r, 
                      how='left',
                      on=key_cols, 
                      suffixes=('_0', '_1')))

In [ ]:
df = df.compute()
df.to_parquet(fn)